In [14]:
import os
import pickle
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import r2_score
from tensorflow import keras
from tensorflow.keras import layers
from tqdm.auto import tqdm, trange

tqdm.pandas()

from collections import defaultdict

local_root_path = "."

sys.path.append(local_root_path)
import annutils

import joblib

In [15]:


# find all the .xlsx files in the local_root_path dir
excel_files = [f for f in os.listdir(local_root_path) if f.endswith(".xlsx") and not f.startswith("~$")]



In order to turn the excel sheets into inputs we can predict on we need to setup

In [16]:
num_sheets = 9

observed_stations_ordered_by_median = ['RSMKL008', 'RSAN032', 'RSAN037', 'RSAC092', 'SLTRM004', 'ROLD024',
                                       'CHVCT000', 'RSAN018', 'CHSWP003', 'CHDMC006', 'SLDUT007', 'RSAN072',
                                       'OLD_MID', 'RSAN058', 'ROLD059', 'RSAN007', 'RSAC081', 'SLMZU025',
                                       'RSAC075', 'SLMZU011', 'SLSUS012', 'SLCBN002', 'RSAC064']

output_stations = ['CHDMC006-CVP INTAKE', 'CHSWP003-CCFB_INTAKE', 'CHVCT000-VICTORIA INTAKE',
                   'OLD_MID-OLD RIVER NEAR MIDDLE RIVER', 'ROLD024-OLD RIVER AT BACON ISLAND',
                   'ROLD059-OLD RIVER AT TRACY BLVD', 'RSAC064-SACRAMENTO R AT PORT CHICAGO',
                   'RSAC075-MALLARDISLAND', 'RSAC081-COLLINSVILLE', 'RSAC092-EMMATON',
                   'RSAC101-SACRAMENTO R AT RIO VISTA', 'RSAN007-ANTIOCH', 'RSAN018-JERSEYPOINT',
                   'RSAN032-SACRAMENTO R AT SAN ANDREAS LANDING', 'RSAN037-SAN JOAQUIN R AT PRISONERS POINT',
                   'RSAN058-ROUGH AND READY ISLAND', 'RSAN072-SAN JOAQUIN R AT BRANDT BRIDGE',
                   'RSMKL008-S FORK MOKELUMNE AT TERMINOUS', 'SLCBN002-CHADBOURNE SLOUGH NR SUNRISE DUCK CLUB',
                   'SLDUT007-DUTCH SLOUGH', 'SLMZU011-MONTEZUMA SL AT BELDONS LANDING',
                   'SLMZU025-MONTEZUMA SL AT NATIONAL STEEL', 'SLSUS012-SUISUN SL NEAR VOLANTI SL',
                   'SLTRM004-THREE MILE SLOUGH NR SAN JOAQUIN R', 'SSS-STEAMBOAT SL', 'CCW-MIDDLE RIVER INTAKE',
                   'OH4-OLD R @ HWY 4', 'SLRCK005-CCWD_Rock', 'MRU-MIDDLE RIVER AT UNDINE ROAD', 'HLL-HOLLAND TRACT',
                   'BET-PIPER SLOUGH @ BETHEL TRACT', 'GES-SACRAMENTO R BELOW GEORGIANA SLOUGH',
                   'NMR: N FORK MOKELUMNE R NEAR WALNUT GROVE', 'IBS-CORDELIA SLOUGH @ IBIS CLUB',
                   'GYS-GOODYEAR SLOUGH AT MORROW ISLAND CLUB', 'BKS-SLBAR002-North Bay Aqueduct/Barker Sl']

output_stations, name_mapping = annutils.read_output_stations(output_stations, observed_stations_ordered_by_median)

In [17]:
def mse_loss_masked(y_true, y_pred):
    squared_diff = tf.reduce_sum(tf.math.squared_difference(y_pred[y_true > 0], y_true[y_true > 0]))
    return squared_diff / (tf.reduce_sum(tf.cast(y_true > 0, tf.float32)) + 0.01)

In [18]:
def predict(location, df_input, output_columns):
    model=keras.models.load_model('%s.h5'%location,custom_objects={"mse_loss_masked": mse_loss_masked})
    xscaler,yscaler=joblib.load('%s_xyscaler.dump'%location)
    return predict_with_model(model, xscaler, yscaler, df_input, output_columns)

def predict_with_model(model, xscaler, yscaler, df_input, output_columns):
    dfx = pd.DataFrame(xscaler.transform(df_input), df_input.index, columns=df_input.columns)

    yyp=model.predict(dfx, verbose=True)
    predicted_y = yscaler.inverse_transform(yyp)
    return pd.DataFrame(predicted_y, index=df_input.index, columns=output_columns)

In [20]:
experiments = [ "colab"]  #"6years", "6yearsAugmented",
for experiment in tqdm(experiments):
    print("Experiment: %s" % experiment)
    experiment_dir = os.path.join(local_root_path, "Experiments", experiment)

    ndays = 118
    window_size = 0
    nwindows = 0

    compression_opts = dict(method='zip', archive_name='out.csv')

    model_dir = os.path.join(experiment_dir, "models")
    model_files = [f for f in os.listdir(model_dir) if f.endswith(".h5")]


    for data_file in tqdm(excel_files):
        print("Data file: %s" % data_file)
        data_path = os.path.join(local_root_path,data_file)
        dfinps, dfouts = annutils.read_and_split(data_path, num_sheets, observed_stations_ordered_by_median)

        dfinps = annutils.create_antecedent_inputs(dfinps,ndays=ndays,window_size=window_size,nwindows=nwindows)
        dfinps, dfouts = annutils.synchronize(dfinps, dfouts)

        #get the name of the file without the extension
        file_name = os.path.splitext(data_file)[0]

        dirs = ["input", "target", "prediction"]
        for dir in dirs:
            os.makedirs(os.path.join("Experiments", experiment, "results", dir), exist_ok=True)

        input_file = os.path.join("Experiments", experiment, "results", "input", file_name + ".csv")
        dfinps.to_csv(input_file, compression=compression_opts)

        # read_in = pd.read_csv(input_file, compression=compression_opts, index_col=0)

        target_file = os.path.join("Experiments", experiment, "results", "target", file_name + "_target.csv")
        dfouts.to_csv(target_file, compression=compression_opts)

        for model_file in tqdm(model_files):
            print("Model file: %s" % model_file)
            model_name = os.path.splitext(model_file)[0]

            model_prediction_dir = os.path.join(experiment_dir, "results", "prediction", model_name)
            os.makedirs(model_prediction_dir, exist_ok=True)

            location = os.path.join(model_dir, model_name)
            print("Location: %s" % location)
            #prediction = predict(location, dfinps, dfouts.columns)
            model=keras.models.load_model('%s.h5'%location,custom_objects={"mse_loss_masked": mse_loss_masked})
            xscaler,yscaler=joblib.load('%s_xyscaler.dump'%location)
            print("Xscaler Min[0]: %s" % xscaler.min_val[0])
            print("Xscaler Max[0]: %s" % xscaler.max_val[0])
            scaled_input = xscaler.transform(dfinps)
            dfx = pd.DataFrame(scaled_input, dfinps.index, columns=dfinps.columns)

            yyp=model.predict(dfx, verbose=True)
            predicted_y = yscaler.inverse_transform(yyp)
            prediction = pd.DataFrame(predicted_y, index=dfinps.index, columns=dfouts.columns)
            prediction_file = os.path.join(model_prediction_dir, file_name + ".csv")
            prediction.to_csv(prediction_file, compression=compression_opts)

print("Done!")

  0%|          | 0/1 [00:00<?, ?it/s]

Experiment: colab


  0%|          | 0/14 [00:00<?, ?it/s]

Data file: dsm2_ann_inputs_base.xlsx


  0%|          | 0/5 [00:00<?, ?it/s]

...

Model file: mtl_i118_g8_f_o1.h5
Location: .\Experiments\colab\models\mtl_i118_g8_f_o1
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [==============================] - 1s 3ms/step
Model file: mtl_i118_lstm8_f_o1.h5
Location: .\Experiments\colab\models\mtl_i118_lstm8_f_o1
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [==============================] - 1s 3ms/step
Model file: mtl_i118_residual_gru_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_residual_gru_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [==============================] - 1s 2ms/step
Model file: mtl_i118_residual_lstm_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_residual_lstm_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [==============================] - 1s 2ms/step
Model file: mtl_i118_resnet_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_resnet_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [=

  0%|          | 0/5 [00:00<?, ?it/s]

Model file: mtl_i118_g8_f_o1.h5
Location: .\Experiments\colab\models\mtl_i118_g8_f_o1
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [==============================] - 1s 3ms/step
Model file: mtl_i118_lstm8_f_o1.h5
Location: .\Experiments\colab\models\mtl_i118_lstm8_f_o1
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [==============================] - 1s 3ms/step
Model file: mtl_i118_residual_gru_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_residual_gru_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [==============================] - 1s 2ms/step
Model file: mtl_i118_residual_lstm_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_residual_lstm_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [==============================] - 1s 2ms/step
Model file: mtl_i118_resnet_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_resnet_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
338/338 [=

  0%|          | 0/5 [00:00<?, ?it/s]

Model file: mtl_i118_g8_f_o1.h5
Location: .\Experiments\colab\models\mtl_i118_g8_f_o1
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
225/225 [==============================] - 1s 3ms/step
Model file: mtl_i118_lstm8_f_o1.h5
Location: .\Experiments\colab\models\mtl_i118_lstm8_f_o1
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
225/225 [==============================] - 1s 3ms/step
Model file: mtl_i118_residual_gru_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_residual_gru_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
225/225 [==============================] - 1s 2ms/step
Model file: mtl_i118_residual_lstm_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_residual_lstm_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
225/225 [==============================] - 1s 2ms/step
Model file: mtl_i118_resnet_8_2.h5
Location: .\Experiments\colab\models\mtl_i118_resnet_8_2
Xscaler Min[0]: 3519.98323631286
Xscaler Max[0]: 561182.0
225/225 [=